In [ ]:
import requests
import pandas as pd
import numpy as np
import gmaps
import json
import csv
from keys import gapikey
from matplotlib.cm import winter
from matplotlib.colors import to_hex

gmaps.configure(api_key=gapikey)

with open('zcta/zcta.json') as f:
    geoj = json.load(f)

with open('resources/zip_targets.csv', newline='\n', encoding='utf-8-sig') as e:
    ziptgt = pd.read_csv(e)

with open('resources/dc_pop.csv', newline='\n', encoding='utf-8-sig') as f:
    dc_popdf = pd.read_csv(f)

In [ ]:
dc_zippop = dc_popdf[['GEO.id2','HC01_EST_VC01']]
dc_zippop = dc_zippop.rename(columns={'GEO.id2': 'zip', 'HC01_EST_VC01': 'population'})
zippopdf = ziptgt.merge(dc_zippop, on='zip', how='outer').set_index('zip')
zipdict = dict(zippopdf['population'])

In [ ]:
poplist = []
geoj_scrubbed = {}
features = geoj['features']
feature_col = []

for feature in features:
    try:
        feature_geoid = int(feature['properties']['GEOID10'])
        if feature_geoid in zipdict.keys():
            feature_col.append(feature)
            poplist.append(zipdict[feature_geoid])
    except ValueError:
            print(f'Value Error {b}')
                  
geoj_scrubbed = {'type': 'FeatureCollection', 'features': feature_col}

In [ ]:
min_pop = min(poplist)
max_pop = max(poplist)
pop_range = max_pop - min_pop

def calculate_color(pop):
    # make pop a number between 0 and 1
    normalized_pop = (pop - min_pop) / pop_range
    # optional invert pop so that high inequality can give dark color
    inverse_pop = 1.0 - normalized_pop
    # transform the pop value to a matplotlib color
    mpl_color = winter(normalized_pop)
    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    return gmaps_color

colors = []
for pop in poplist:
    if str(pop) != 'nan':
        try:
            color = calculate_color(pop)
        except KeyError:
            color = (0, 0, 0, 0.3)
    else:
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [ ]:
zcta_layer = gmaps.geojson_layer(
    geoj_scrubbed,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig = gmaps.figure()
fig.add_layer(zcta_layer)
fig